
 Student Name: Matthew Cheng

 Student ID: 107990014
<br /><br />
Due Date: November 23, 2021

In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statistics
import pprint

from operator import add
from sklearn import preprocessing
from collections import Counter

In [28]:
!mkdir p1
!mkdir p2
!mkdir p3
!mkdir p4




mkdir: cannot create directory ‘p1’: File exists
mkdir: cannot create directory ‘p2’: File exists
mkdir: cannot create directory ‘p3’: File exists
mkdir: cannot create directory ‘p4’: File exists


In [29]:
!unzip /content/hw2_data.zip -d data

Archive:  /content/hw2_data.zip
replace data/Facebook_Economy.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [30]:
data = pd.read_csv('/content/data/News_Final.csv')
df = pd.DataFrame(data, dtype=None)
df.replace('?', np.nan,inplace = True)

for col in df.columns:
  df[col].fillna('', inplace=True)
  if df[col].dtype == object and col != 'PublishDate':
    df[col] = df[col].str.replace('[^\w\s]','') #remove punctuation


df = df.applymap(lambda s: s.lower() if type(s) == str else s)
df.isnull().sum()

IDLink               0
Title                0
Headline             0
Source               0
Topic                0
PublishDate          0
SentimentTitle       0
SentimentHeadline    0
Facebook             0
GooglePlus           0
LinkedIn             0
dtype: int64

In [31]:
df

,IDLink,Title,Headline,Source,Topic,PublishDate,SentimentTitle,SentimentHeadline,Facebook,GooglePlus,LinkedIn
0,99248.0,obama lays wreath at arlington national cemetery,obama lays wreath at arlington national cemete...,usa today,obama,2002-04-02 00:00:00,0.000000,-0.053300,-1,-1,-1
1,10423.0,a look at the health of the chinese economy,tim haywood investment director businessunit h...,bloomberg,economy,2008-09-20 00:00:00,0.208333,-0.156386,-1,-1,-1
2,18828.0,nouriel roubini global economy not back to 2008,nouriel roubini nyu professor and chairman at ...,bloomberg,economy,2012-01-28 00:00:00,-0.425210,0.139754,-1,-1,-1
3,27788.0,finland gdp expands in q4,finlands economy expanded marginally in the th...,rtt news,economy,2015-03-01 00:06:00,0.000000,0.026064,-1,-1,-1
4,27789.0,tourism govt spending buoys thai economy in ja...,tourism and public spending continued to boost...,the nation thailand39s english news,economy,2015-03-01 00:11:00,0.000000,0.141084,-1,-1,-1
...,...,...,...,...,...,...,...,...,...,...,...
93234,61851.0,stocks rise as investors key in on us economy ...,the june employment report is viewed as a cruc...,marketwatch,economy,2016-07-07 15:31:05,0.104284,0.044943,-1,3,5
93235,61865.0,russian pm proposes to use conservative and to...,in addition establish stimulating economic pol...,tass,economy,2016-07-07 15:31:10,0.072194,0.000000,-1,0,1
93236,104793.0,palestinian government uses foreign aid to pay...,the palestinian government spends nearly 140 m...,daily caller,palestine,2016-07-07 15:38:26,0.291667,-0.139754,5,1,0
93237,104794.0,palestine youth orchestra prepares for first u...,palestine youth orchestra prepares for first u...,ahram online,palestine,2016-07-07 15:59:22,0.121534,0.092313,0,0,0


### Part (1):    
In news data, count the words in two fields: ‘Title’ and ‘Headline’ respectively, and list the most frequent words according to the term frequency in descending order, in total, per day, and per topic, respectively


In [32]:
df_titlefreq = df.Title.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count')
df_titlefreq.to_csv('p1/freq_title_total.csv')

df_titlefreq

,word,count
0,to,27453
1,economy,26240
2,obama,22583
3,the,17738
4,microsoft,17598
...,...,...
35746,witty,1
35747,dampb,1
35748,grunke,1
35749,yahyah,1


In [33]:
df_headlinefreq = df.Headline.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count')
df_headlinefreq.to_csv('p1/freq_headline_total.csv')

df_headlinefreq

,word,count
0,the,154328
1,to,69583
2,of,61534
3,a,60208
4,in,54012
...,...,...
62738,osa,1
62739,bseiso,1
62740,suarez,1
62741,sequera,1


In [34]:
topics = ['economy', 'microsoft', 'obama', 'palestine']

for topic in topics:

  temp = topic
  topic = df.loc[df['Topic'] == topic]
  df_p1_title = topic.Title.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count')
  df_p1_headline = topic.Headline.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count')

  df_p1_title.to_csv('p1/freq_title_' + temp + '.csv')
  df_p1_headline.to_csv('p1/freq_headline_' + temp + '.csv')
  

In [35]:
df_day = pd.DataFrame(df, columns = ['PublishDate', 'Title', 'Headline'])
df_day['PublishDate'] = df_day['PublishDate'].str[:10] #get first 10 char of PublishDate
df_day

,PublishDate,Title,Headline
0,2002-04-02,obama lays wreath at arlington national cemetery,obama lays wreath at arlington national cemete...
1,2008-09-20,a look at the health of the chinese economy,tim haywood investment director businessunit h...
2,2012-01-28,nouriel roubini global economy not back to 2008,nouriel roubini nyu professor and chairman at ...
3,2015-03-01,finland gdp expands in q4,finlands economy expanded marginally in the th...
4,2015-03-01,tourism govt spending buoys thai economy in ja...,tourism and public spending continued to boost...
...,...,...,...
93234,2016-07-07,stocks rise as investors key in on us economy ...,the june employment report is viewed as a cruc...
93235,2016-07-07,russian pm proposes to use conservative and to...,in addition establish stimulating economic pol...
93236,2016-07-07,palestinian government uses foreign aid to pay...,the palestinian government spends nearly 140 m...
93237,2016-07-07,palestine youth orchestra prepares for first u...,palestine youth orchestra prepares for first u...


In [36]:
for date, df_date in df_day.groupby('PublishDate'):
    curr_df = df_date.Title.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count') #the df for DATE
    curr_df.to_csv('p1/freq_title_' + date + '.csv')
    curr_df = df_date.Headline.str.split(expand=True).stack().value_counts().rename_axis('word').reset_index(name='count') #the df for DATE
    curr_df.to_csv('p1/freq_headline_' + date + '.csv')


### Part (2):        
In social feedback data, calculate the average popularity of each news by hour, and by day, respectively (for each platform)


In [37]:
dir_topic = ['Economy', 'Microsoft', 'Obama', 'Palestine'] #the four directories of each csv
dir_platform = ['Facebook', 'GooglePlus', 'LinkedIn'] #platforms


for platform in dir_platform:
  hr_avg = []
  day_avg = []
  
  for topic in dir_topic:
    
    dir = '/content/data/' + platform + '_' + topic + '.csv' 
    print(dir) 

    data = pd.read_csv(dir) 
    df_topic = pd.DataFrame(data, dtype=None) #create the df for topic
    df.replace('?', np.nan,inplace=True)

    df_ts = df_topic['TS144'].values
    print(len(df_ts))

    for i in range(0,len(df_ts)): #get avg popularity of every news article
      hr_avg.append(df_ts[i].sum()/48)
      day_avg.append(df_ts[i].sum()/2)

  #after all topics for current platform have been looped, create a dataframe of the hr and day avgs 
  df_hr = pd.DataFrame(hr_avg) 
  df_day = pd.DataFrame(day_avg)

  df_hr.to_csv('p2/pop_' + platform + '_hr_avg.csv')
  df_day.to_csv('p2/pop_' + platform + '_day_avg.csv')
  

/content/data/Facebook_Economy.csv
29928
/content/data/Facebook_Microsoft.csv
18531
/content/data/Facebook_Obama.csv
27015
/content/data/Facebook_Palestine.csv
7687
/content/data/GooglePlus_Economy.csv
33069
/content/data/GooglePlus_Microsoft.csv
20702
/content/data/GooglePlus_Obama.csv
27157
/content/data/GooglePlus_Palestine.csv
7749
/content/data/LinkedIn_Economy.csv
33069
/content/data/LinkedIn_Microsoft.csv
20702
/content/data/LinkedIn_Obama.csv
27157
/content/data/LinkedIn_Palestine.csv
7749


### Part (3):           
In news data, calculate the sum and average sentiment score of each topic, respectively

In [38]:
dftitle = df[['Topic','SentimentTitle','SentimentHeadline']].copy() #df with only title 
dftitle


,Topic,SentimentTitle,SentimentHeadline
0,obama,0.000000,-0.053300
1,economy,0.208333,-0.156386
2,economy,-0.425210,0.139754
3,economy,0.000000,0.026064
4,economy,0.000000,0.141084
...,...,...,...
93234,economy,0.104284,0.044943
93235,economy,0.072194,0.000000
93236,palestine,0.291667,-0.139754
93237,palestine,0.121534,0.092313


In [39]:
topics = ['economy', 'microsoft', 'obama', 'palestine']
output_values = []

for topic in topics:

  temp = topic
  topic = dftitle.loc[dftitle['Topic'] == topic]
  output_values.append(topic['SentimentTitle'].sum())
  output_values.append(topic['SentimentTitle'].sum()/len(topic))
  output_values.append(topic['SentimentHeadline'].sum())
  output_values.append(topic['SentimentHeadline'].sum()/len(topic))


for i in output_values:
  print(i)

-350.89723953455336
-0.010342408616321427
-1340.1388172155666
-0.03949949355150809
51.56650733416538
0.002359159453479979
-322.2826752054226
-0.014744380785315335
-27.604435365771053
-0.0009648526866749757
-507.62019742134817
-0.01774275419158854
-177.6151443106848
-0.020085394584494494
-393.38238008336015
-0.044485172462214194


### Part 4:
From subtask (1), for the top-100 frequent words per topic in titles and headlines, calculate their co-occurrence matrices (100x100), respectively. Each entry in the matrix will contain the co-occurrence frequency in all news titles and headlines, respectively 

In [40]:
window_len = 1
def update_co_mat(x, co_mat, unique):   
    # get words in sentence into array
    w_list = x.split(' ')
    # focus each word
    for focus_w_index, focus_w in enumerate(w_list):
        focus_w = focus_w.lower()
        # Get the indices of all the context words for the given focus word
        for context_w_index in range(max(0,focus_w_index - window_len),min(len(w_list),focus_w_index + window_len + 1)):   #0;1, 1;2, ... ,len-1;len                      
            # If context words are in the unique words list
            if w_list[context_w_index] in unique and focus_w in unique:
                # To identify the row number, get the index of the focus_w in unique list
                co_mat_row_index = unique.index(focus_w)
                
                # To identify the column number, get the index of the neighbor word in the unique list
                co_mat_col_index = unique.index(w_list[context_w_index])  
                                
                # Update the respective columns of the corresponding focus word row
                if co_mat_row_index < 100 and co_mat_col_index < 100:
                  co_mat[co_mat_row_index][co_mat_col_index] += 1

In [41]:
topics = ['economy', 'microsoft', 'obama', 'palestine']
df_p4 = pd.DataFrame(df, columns = ['Topic', 'Title', 'Headline'])

freq_matrix = []

for df_t in topics:
  df_t = df_p4.loc[df_p4['Topic'] == df_t].Title.str.split(expand=True).stack().value_counts()\
              .rename_axis('word').reset_index(name='count')\
              .head(100)['word'].tolist() #get 100 most frequent words for topic's titles
  freq_matrix.append(df_t)

for df_t in topics:
  df_t = df_p4.loc[df_p4['Topic'] == df_t].Headline.str.split(expand=True).stack().value_counts()\
              .rename_axis('word').reset_index(name='count')\
              .head(100)['word'].tolist() #get 100 most frequent words for topic's headlines
  freq_matrix.append(df_t)

for row in freq_matrix:
  print(row) 

['economy', 'the', 'to', 'in', 'of', 'for', 'on', 'us', 'economic', 'a', 'is', 'as', 'and', 'global', 'growth', 'says', 'china', 'will', '2016', 'chinas', 'new', 'at', 'brexit', 'about', 'by', 'with', 'how', 'uk', 'be', 'could', 'boost', 'more', 'not', 'but', 'up', 'world', 'quarter', 'oil', 'its', 'are', 'than', 'from', 'bank', 'jobs', 'minister', 'rate', 'over', 'year', 'why', 'can', 'may', '2015', 'grow', 'what', 'market', 'report', 'fed', 'chinese', 'has', 'sharing', 'growing', 'business', 'eu', 'india', 'this', 'grows', 'gdp', 'percent', 'would', 'state', 'still', 'after', 'recession', 'data', 'despite', 'it', 'good', 'first', 'help', 'an', 'into', 'russian', 'weak', 'japan', 'no', 'stocks', 'imf', 'warns', 'german', 'that', 'grew', 'shows', 'strong', 'expected', 'prices', 'fuel', 'have', 'outlook', 'impact', 'down']
['microsoft', 'to', 'for', 'the', 'microsofts', 'and', 'in', 'windows', 'with', 'a', 'on', 'of', '10', 'is', 'new', 'surface', 'xbox', 'its', 'cloud', 'one', 'office'

In [42]:
df_p4.loc[df_p4['Topic'] == 'economy'].Title.str.split(expand=True).stack().value_counts()\
              .rename_axis('word').reset_index(name='count').head(10)

,word,count
0,economy,26121
1,the,8976
2,to,8834
3,in,6854
4,of,4059
5,for,3982
6,on,3601
7,us,2880
8,economic,2792
9,a,2710


In [43]:
#the actual code
#note: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0151146

topics = ['economy', 'microsoft', 'obama', 'palestine']
freqs = 100

for i in range(8):
  u = freq_matrix[i] #the top freqs for current topic, title/headline

  comat = [[0 for i in range(freqs)] for j in range(freqs)]

  df_curr = df_p4.loc[df_p4['Topic'] == topics[i % 4]]
  
  col_curr = 'Title' if i < 4 else 'Headline' 

  curr = topics[i % 4] + '_' + col_curr.lower()

  print('Processing ' + curr)
  for index, row in df_curr.iterrows():
     update_co_mat(row[col_curr], comat, u)

  print('Co-occurence matrix of ' + curr + ': ')
  for row in comat:
    print(row)
  pd.DataFrame(comat).to_csv('p4/comat_' + curr + '.csv')


Processing economy_title
Co-occurence matrix of economy_title: 
[26120, 2100, 943, 932, 288, 396, 768, 1672, 4, 107, 946, 634, 461, 1254, 26, 375, 198, 252, 40, 512, 147, 137, 26, 123, 158, 152, 24, 520, 8, 131, 273, 44, 144, 114, 60, 391, 1, 52, 164, 31, 45, 89, 5, 42, 350, 4, 121, 4, 7, 125, 136, 17, 55, 20, 116, 56, 19, 266, 140, 415, 164, 9, 34, 19, 59, 362, 7, 4, 47, 81, 132, 82, 5, 24, 36, 4, 30, 9, 54, 75, 62, 184, 118, 106, 21, 11, 13, 33, 246, 50, 296, 123, 74, 50, 5, 256, 7, 15, 22, 36]
[2100, 8974, 291, 556, 578, 588, 459, 564, 88, 0, 256, 34, 331, 439, 8, 25, 11, 39, 29, 0, 121, 55, 30, 329, 25, 55, 137, 116, 30, 12, 49, 1, 51, 13, 36, 177, 0, 18, 56, 42, 24, 78, 7, 8, 4, 1, 32, 26, 122, 30, 0, 6, 31, 49, 25, 2, 76, 80, 28, 229, 17, 9, 43, 10, 4, 0, 2, 0, 3, 49, 10, 10, 8, 6, 7, 5, 26, 21, 34, 0, 45, 18, 9, 0, 4, 3, 10, 1, 5, 25, 1, 7, 3, 1, 0, 4, 11, 10, 31, 19]
[943, 291, 8834, 3, 1, 0, 9, 74, 42, 113, 21, 1, 0, 89, 93, 3, 54, 2, 25, 9, 32, 0, 30, 19, 0, 1, 124, 39, 218, 

### Downloading The Outputs

In [45]:
!zip -r /content/p1.zip /content/p1
!zip -r /content/p2.zip /content/p2
!zip -r /content/p3.zip /content/p3
!zip -r /content/p4.zip /content/p4

from google.colab import files
files.download("/content/p1.zip")
files.download("/content/p2.zip")
files.download("/content/p3.zip")
files.download("/content/p4.zip")

  adding: content/p1/ (stored 0%)
  adding: content/p1/freq_headline_microsoft.csv (deflated 58%)
  adding: content/p1/freq_headline_2016-06-29.csv (deflated 56%)
  adding: content/p1/freq_headline_2016-06-06.csv (deflated 56%)
  adding: content/p1/freq_title_2016-04-07.csv (deflated 54%)
  adding: content/p1/freq_title_2016-06-06.csv (deflated 54%)
  adding: content/p1/freq_headline_2015-12-27.csv (deflated 54%)
  adding: content/p1/freq_title_2016-02-29.csv (deflated 53%)
  adding: content/p1/freq_title_2016-04-24.csv (deflated 52%)
  adding: content/p1/freq_title_2015-12-26.csv (deflated 52%)
  adding: content/p1/freq_title_2015-11-26.csv (deflated 53%)
  adding: content/p1/freq_title_2016-05-22.csv (deflated 52%)
  adding: content/p1/freq_headline_2016-04-28.csv (deflated 56%)
  adding: content/p1/freq_headline_2016-03-08.csv (deflated 56%)
  adding: content/p1/freq_headline_2016-01-25.csv (deflated 56%)
  adding: content/p1/freq_title_2016-05-09.csv (deflated 54%)
  adding: conten

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>